# 2 mini graficos. Size org y Employment

In [1]:
import pandas as pd

import plotly.express as px

import dash
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc #bootstrap


#crear un dataframe con toda la informacion de la encuesta
df21 = pd.read_csv('../data/survey_results_public2021.csv', engine="c",
                  usecols= ["MainBranch","Country","US_State","EdLevel", "Age", "Employment",
                   "Age1stCode","LearnCode","YearsCode","YearsCodePro","DevType","OpSys","NEWStuck",
               "ConvertedCompYearly", "LanguageHaveWorkedWith", "LanguageWantToWorkWith","DatabaseHaveWorkedWith",
               "DatabaseWantToWorkWith", "PlatformHaveWorkedWith","PlatformWantToWorkWith",
               "WebframeHaveWorkedWith", "WebframeWantToWorkWith","MiscTechHaveWorkedWith",
               "MiscTechWantToWorkWith", "ToolsTechHaveWorkedWith","ToolsTechWantToWorkWith",
               'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', "ConvertedCompYearly", "OrgSize"]) # El indice sera la columna con el ID de la respuesta

#pd.set_option("display.max_rows", None) #permite ver el output completo de la celda


# 1. procesar datos

In [2]:
def graficosRespuesta(df, col):

    df=df21[[col]].dropna().copy() #copia df

    df2 = (df.explode(df.columns.tolist())
           .apply(lambda col: col.str.split(';')) 
           .explode(col).groupby(col)
           .agg(responses=(col,'count')) 
           .reset_index())
    
    #eliminar todos los valores que contengan ciertos strings
    # ej: Other (please specify), Something else... 
    df2 = df2[~df2[col].str.contains('|'.join(['please','not to say', 'else', 't know']))] 
    
    #eliminar la info dentro de parentesis 
    df2[col] = df2[col].str.replace(r"\(.*?\)", "", regex=True) 
    
    #ordenar de mas a menos respuestas
    df2.sort_values(["responses"], ascending=True, inplace = True)

    return df2

# 2. Layout

Si no se va a añadir ningun elemento interactivo (ej dropdown o botones) se puede anyadir el gráfico directamente en el layout sin usar callbacks

In [3]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport', #permite ser responsive en movil
                            'content': 'width=device-width, initial-scale=1.0, maximum-scale=1.2, minimum-scale=0.5,'}]
                )

app.layout = html.Div([
    
    dbc.Row([ 
            dbc.Col([dcc.Loading([ 
                dcc.Graph(id = 'OrgSize', figure = px.pie(graficosRespuesta(df21, "OrgSize"), 
                                                        values='responses', names="OrgSize",
                                                       title = "Company size",
                                                       color_discrete_sequence = px.colors.qualitative.Safe,
                                                       labels={"responses": "# Responses", 
                                                              "OrgSize": "Org size"})
                         ), #stuck
                ]), #spinner
            ]),#col
            dbc.Col([dcc.Loading([
                        dcc.Graph(id = 'Employment', figure  =
                            (px.bar(graficosRespuesta(df21, "Employment"), y='Employment', x= 'responses',
                                    orientation = "h",text_auto= True, title = "Employment status", 
                                    color_discrete_sequence = px.colors.qualitative.Safe,
                                   labels={"Age1stCode": "Age start coding", "responses": "# Responses"}))
                                  .update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
                                  .update_layout(yaxis_title=None)
                                 
                                 ), #Age1stCode 
                        ]), #spinner
                    ]),#col
        ]), #row
                    

            ]) #div




In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
